In [1]:
from transformers import AutoTokenizer
from transformers import TrainingArguments
from transformers import AdapterTrainer
from datasets import load_dataset, load_from_disk
from transformers import AutoModelForQuestionAnswering
from torch.utils.data import DataLoader
from transformers import default_data_collator
import evaluate
from tqdm import tqdm
from transformers import pipeline

/home/mmm/anaconda3/envs/aml/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [3]:
max_length = 512
stride = 128

def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    context = examples["context"]
    
    # Tokenize question and context together into one input,
    # they will seperable with a special token between them.
    # Tokenizer will also split context into multiple chuncks,
    # if the max_length is exceeded.
    inputs = tokenizer(
        questions,
        context,
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # offset_mapping is the result of the split into
    # multiple chunks
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    # This step is required to see if an answer is inside
    # the context chunks or not. It labels the multiple chunks
    # generated by the tokenizer into either not having
    # the answer, or where the answer is located
    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [4]:
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    context = examples["context"]
    
    # It's the same thing as with the preprocess_training_examples
    # tokenizer, but here 
    inputs = tokenizer(
        questions,
        context,
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [3]:
langs = ["en", "es", "de", "el", "ru", "tr", "ar", "vi", "zh", "hi", "ro", "th"]

In [4]:
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-bas

In [10]:
batch_size = 8
epochs = 20

In [11]:
for lang in langs:
    train = load_from_disk(f'../data/xquad_{lang}_train.hf/')
    val = load_from_disk(f'../data/xquad_{lang}_val.hf/')
    train_dataset = train.map(
        preprocess_training_examples,
        batched=True,
        remove_columns=train.column_names,
    )

    validation_dataset = val.map(
        preprocess_validation_examples,
        batched=True,
        remove_columns=val.column_names,
    )
    train_dataset.set_format("torch")
    validation_set = validation_dataset.remove_columns(["example_id", "offset_mapping"])
    validation_set.set_format("torch")

    train_dataloader = DataLoader(
        train_dataset,
        shuffle=True,
        collate_fn=default_data_collator,
        batch_size=8,
    )

    validation_dataloader = DataLoader(
        validation_dataset,
        shuffle=True,
        collate_fn=default_data_collator,
        batch_size=8,
    )
    adapter_name = f'xquad_adapter'
    model.add_adapter(adapter_name)
    # model.load_adapter("../adapter_weights/adapter_qa_en_mbert_4ep/")
    model.train_adapter(adapter_name)
    model.set_active_adapters(adapter_name)
    model_name = model_checkpoint.split("/")[-1]
    args = TrainingArguments(
        f"{model_name}-adapter-xquad",
        evaluation_strategy = "epoch",
        learning_rate=1e-4,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=0.01
    )
    trainer = AdapterTrainer(
        model,
        args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        data_collator=default_data_collator,
        tokenizer=tokenizer
    )
    trainer.train()
    model.save_adapter(f"../adapter_qa_{lang}_mbert_{epochs}", adapter_name=adapter_name)
    model.set_active_adapters(None)
    model.delete_adapter(adapter_name)
    del train, val

Loading cached processed dataset at ../data/xquad_en_train.hf/cache-c1c761e0b7b0eb11.arrow
Loading cached processed dataset at ../data/xquad_en_val.hf/cache-3ca8755dd58ddcef.arrow
Adding adapter 'xquad_adapter'.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/home/mmm/anaconda3/envs/aml/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 727
  Num Epochs = 20
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & ac

{'eval_runtime': 20.4812, 'eval_samples_per_second': 35.496, 'eval_steps_per_second': 4.443, 'epoch': 1.0}


 10%|█         | 182/1820 [01:54<13:36,  2.01it/s] The following columns in the evaluation set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Evaluation *****
  Num examples = 727
  Batch size = 8

 10%|█         | 182/1820 [02:15<13:36,  2.01it/s]

{'eval_runtime': 21.2761, 'eval_samples_per_second': 34.17, 'eval_steps_per_second': 4.277, 'epoch': 2.0}


 15%|█▌        | 273/1820 [03:03<12:51,  2.01it/s]  The following columns in the evaluation set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Evaluation *****
  Num examples = 727
  Batch size = 8

 15%|█▌        | 273/1820 [03:23<12:51,  2.01it/s]

...

{'loss': 0.5444, 'learning_rate': 6.015037593984962e-06, 'epoch': 18.8}


 95%|█████████▌| 2527/2660 [47:44<01:21,  1.64it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Evaluation *****
  Num examples = 1057
  Batch size = 8

 95%|█████████▌| 2527/2660 [48:34<01:21,  1.64it/s]

{'eval_runtime': 50.6597, 'eval_samples_per_second': 20.865, 'eval_steps_per_second': 2.625, 'epoch': 19.0}


100%|██████████| 2660/2660 [50:25<00:00,  1.56it/s]The following columns in the evaluation set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: offset_mapping, example_id.
***** Running Evaluation *****
  Num examples = 1057
  Batch size = 8

100%|██████████| 2660/2660 [51:17<00:00,  1.56it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 2660/2660 [51:17<00:00,  1.16s/it]
Configuration saved in ../adapter_qa_th_mbert_20/adapter_config.json
Module weights saved in ../adapter_qa_th_mbert_20/pytorch_adapter.bin
Configuration saved in ../adapter_qa_th_mbert_20/head_config.json
Module weights saved in ../adapter_qa_th_mbert_20/pytorch_model_head.bin


{'eval_runtime': 51.7018, 'eval_samples_per_second': 20.444, 'eval_steps_per_second': 2.572, 'epoch': 20.0}
{'train_runtime': 3077.3133, 'train_samples_per_second': 6.87, 'train_steps_per_second': 0.864, 'train_loss': 0.9997729136531515, 'epoch': 20.0}


In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
# model_checkpoint = adapter_checkpoint
question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer)

context = """
? Transformers is backed by the three most popular deep learning libraries ? Jax, PyTorch and TensorFlow ? with a seamless integration
between them. It's straightforward to train your models with one before loading them for inference with the other.
"""
question = "What is backed by deep learning libraries?"
question_answerer(question=question, context=context)

In [5]:
def get_predictions(dataset):
    
    predictions = []
    for example in tqdm(dataset):
        question = example['question']
        context = example['context']
        prediction = question_answerer(question=question, context=context)

        predictions.append(prediction)
    
    return predictions

In [6]:
# Need to convert the variables so that they can be used by the evaluation.compute function
def convert_for_evaluation(predictions, examples):
    ref = []
    pred = []
    for i, id in enumerate(examples['id']):
        ref.append({
            'answers': examples['answers'][i],
            'id': examples['id'][i]
        })
        pred.append({
            'prediction_text': predictions[i]['answer'],
            'id': examples['id'][i]
        })
        
    return pred, ref

In [7]:
squad_metric = evaluate.load("squad")
results_en2lang = {}
results_en = {}

for lang in langs:
    test = load_from_disk(f'../data/xquad_{lang}_test.hf/')

    adapter_name = f'xquad_adapter'
    model.load_adapter(f"../adapter_weights/adapter_qa_{lang}_mbert_20")
    model.train_adapter(adapter_name)
    model.set_active_adapters(adapter_name)

    question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer)
    print(f'Running predictions for {lang}')
    predictions = get_predictions(test)

    predictions, references = convert_for_evaluation(predictions, test) 
    res = squad_metric.compute(predictions=predictions, references=references)
    results_en2lang[lang] = res

Running predictions for en


  8%|▊         | 20/238 [00:04<00:39,  5.51it/s]/home/mmm/anaconda3/envs/aml/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:708: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/home/mmm/anaconda3/envs/aml/lib/python3.10/site-packages/transformers/pipelines/question_answering.py:298: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  p_mask = np.asarray(
100%|██████████| 238/238 [00:49<00:00,  4.77it/s]
Overwriting existing adapter 'xquad_adapter'.


Running predictions for es


100%|██████████| 238/238 [01:02<00:00,  3.78it/s]
Overwriting existing adapter 'xquad_adapter'.


Running predictions for de


...

100%|██████████| 238/238 [01:36<00:00,  2.46it/s]
Overwriting existing adapter 'xquad_adapter'.


Running predictions for ro


100%|██████████| 238/238 [01:18<00:00,  3.01it/s]
Overwriting existing adapter 'xquad_adapter'.


Running predictions for th


100%|██████████| 238/238 [03:20<00:00,  1.19it/s]


In [8]:
results_en2lang

{'en': {'exact_match': 42.016806722689076, 'f1': 52.958167338419464},
 'es': {'exact_match': 29.831932773109244, 'f1': 51.29742386700498},
 'de': {'exact_match': 34.45378151260504, 'f1': 46.94773980273347},
 'el': {'exact_match': 31.092436974789916, 'f1': 43.02789647327463},
 'ru': {'exact_match': 34.87394957983193, 'f1': 48.446618379755726},
 'tr': {'exact_match': 25.630252100840337, 'f1': 39.659607432716676},
 'ar': {'exact_match': 26.050420168067227, 'f1': 41.64258127493421},
 'vi': {'exact_match': 32.773109243697476, 'f1': 52.76812381551958},
 'zh': {'exact_match': 31.092436974789916, 'f1': 40.01400560224091},
 'hi': {'exact_match': 31.932773109243698, 'f1': 46.186680435967574},
 'ro': {'exact_match': 37.39495798319328, 'f1': 53.1701333820386},
 'th': {'exact_match': 36.134453781512605, 'f1': 41.9467787114846}}